### Create new price list

In [136]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
from os.path import join

load_folder = 'c:/Users/balob/Documents/GITLAB/RB_BD/DATA/'

eur_usd = 1.2

roaming_plan_id_s1 = 267
roaming_plan_id_s2 = 268

In [137]:
# Из базы данных OCS выгружаем TADIG

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT DISTINCT c.COUNTRY_NAME,n.NETWORK_NAME,t.TADIG_CODE,t.NETWORK_ID,
np.MCC + IIF(LEN(MNC)=1,'0' + MNC,MNC) MCCMNC
FROM RDB_TADIG_CODES t
LEFT JOIN RDB_NETWORKS n ON t.network_id = n.network_id
LEFT JOIN RDB_COUNTRIES c ON n.country_id = c.country_id
LEFT JOIN RDB_NETWORK_IMSI_PREFIXES np ON t.network_id = np.network_id
'''
df_tadig_mccmnc = pd.read_sql_query(sql_srt, cnxn)
#df_tadig['NETWORK_ID']=df_tadig['NETWORK_ID'].astype('int')
df_tadig = df_tadig_mccmnc[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','NETWORK_ID']].drop_duplicates()

df_tadig_mccmnc = df_tadig_mccmnc[['NETWORK_ID','MCCMNC']].drop_duplicates()
display(df_tadig.head())

sql_srt=\
'''
SELECT * FROM ROAMING_PLAN_RULES r
WHERE r.roaming_plan_id IN (267,268)
AND r.end_date IS NULL
'''

df_roaming_plan_rules = pd.read_sql_query(sql_srt, cnxn)
df_roaming_plan_rules_s1 = df_roaming_plan_rules[df_roaming_plan_rules.ROAMING_PLAN_ID == roaming_plan_id_s1]

df_roaming_plan_rules = pd.read_sql_query(sql_srt, cnxn)
df_roaming_plan_rules_s2 = df_roaming_plan_rules[df_roaming_plan_rules.ROAMING_PLAN_ID == roaming_plan_id_s2]

display(df_roaming_plan_rules_s1.head())
display(df_roaming_plan_rules_s2.head())

cnxn.close()

,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
0,Afghanistan,Afghan Wireless Communication Company,AFGAW,4.0
1,Afghanistan,Areeba/MTN,AFGAR,5.0
2,Afghanistan,Etisalat,AFG55,6.0
3,Afghanistan,Etisalat,AFGEA,6.0
4,Afghanistan,Roshan/TDCA,AFGK9,7.0


,ROAMING_PLAN_RULE_ID,ROAMING_PLAN_ID,NETWORK_ID,SPONSOR_ID,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,MT_SMS_RATE,DATA_RATE,CURRENCY_ID,START_DATE,END_DATE,is_discounted
117,5567.0,267.0,59.0,1.0,0.990086,0.990086,0.249622,0.0,10.238187,1.0,2017-11-01,None,False
118,5568.0,267.0,95.0,1.0,2.189060,2.189060,0.450132,0.0,14.333252,1.0,2017-11-01,None,False
119,5570.0,267.0,108.0,1.0,0.660066,0.000000,0.299730,0.0,9.997989,1.0,2017-11-01,None,False
120,5571.0,267.0,113.0,1.0,2.367968,0.000000,0.494058,0.0,15.786522,1.0,2017-11-01,None,False
121,5600.0,267.0,325.0,1.0,0.599000,0.000000,0.199000,0.0,6.998000,1.0,2018-02-01,None,False


,ROAMING_PLAN_RULE_ID,ROAMING_PLAN_ID,NETWORK_ID,SPONSOR_ID,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,MT_SMS_RATE,DATA_RATE,CURRENCY_ID,START_DATE,END_DATE,is_discounted
0,4374.0,268.0,10.0,2.0,0.468,0.0,0.0480,0.0,0.0468,1.0,2017-12-01,None,True
1,4375.0,268.0,30.0,2.0,0.702,0.0,0.0180,0.0,0.0048,1.0,2017-12-01,None,True
2,4376.0,268.0,45.0,2.0,0.066,0.0,0.0096,0.0,0.0096,1.0,2017-12-01,None,True
3,4377.0,268.0,50.0,2.0,0.042,0.0,0.0156,0.0,0.0096,1.0,2017-12-01,None,True
4,4378.0,268.0,51.0,2.0,0.042,0.0,0.0156,0.0,0.0096,1.0,2017-12-01,None,True


In [138]:
# Загружаем новые тарифы Partner и P4

df_partner = pd.read_csv(join(load_folder,'Partner_1_1_2019_for_tariffs.csv'),sep=';')
print('Тарифы Partner:')
display(df_partner.head())

df_p4 = pd.read_csv(join(load_folder,'P4_1_1_2019_for_tariffs.csv'),sep=';')
print('Тарифы P4:')
display(df_p4.head())

Тарифы Partner:


,TADIG,MCCMNC,PARTNER,COUNTRY,Operator,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB
0,ALBVF,27602.0,Vodafone AL,Albania,VF,0.0439,0.0,0.0062,0.0030,1,1.0
1,ANDMA,21303.0,Andorra Telecom SAU,Andorra,NaN,0.2000,0.0,0.0300,0.2000,60,1.0
2,AIACW,36584.0,AIACW AI,Anguilla,C&W Lime,0.1606,0.0,0.0128,0.0257,60,1.0
3,ATGCW,34492.0,ATGCW AG,Antigua And Barbuda,C&W Lime,0.1500,0.0,0.0120,0.0240,60,1.0
4,ARGTM,72207.0,TELE MOV AR,Argentina,Telefonica LATAM,0.0728,0.0,0.0240,0.0180,1,1.0


Тарифы P4:


,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB
0,Albania,ALBtelecom sh.a.,ALBEM,0.390,NaN,0.040,0.039,60,1
1,Anguilla,Cable and Wireless (Anguilla) Limited,AIACW,0.800,NaN,0.080,0.080,1,1
2,Antigua,Cable & Wireless Antigua and Barbuda Limited,ATGCW,0.800,NaN,0.080,0.080,1,1
3,Armenia,Armenia Telephone Company (ArmenTel),ARM01,0.585,NaN,0.015,0.004,60,1
4,Australia,SingTel Optus Pty Limited,AUSOP,0.055,NaN,0.008,0.008,60,1


In [139]:
# Загружаем Allowed list с необходимыми изменениями

file_allowed_list = 'DMI.dmi_allowed_list_export_Tue_Dec_25_2018.csv'

df_al=pd.read_csv(join(load_folder,file_allowed_list),usecols=['Sponsor','PLMN code'])
df_al.rename({'PLMN code':'TADIG_CODE_AL','Sponsor':'SPONSOR'},inplace=True,axis=1)
df_al = df_al[df_al.SPONSOR.isin(['Partner_Combined','P4_Combined'])]
df_al = df_al.replace({'Partner_Combined':'S1','P4_Combined':'S2'})
df_al = pd.merge(df_al,df_tadig[['TADIG_CODE','NETWORK_ID']],how='left',left_on='TADIG_CODE_AL',right_on='TADIG_CODE')
df_al.drop('TADIG_CODE',inplace=True,axis=1)

# Add PLMN

df_add_plmn = DataFrame({'SPONSOR':['S1'],'TADIG_CODE_AL':['RUSNW'],'NETWORK_ID':[769]})
df_al = df_al.append(df_add_plmn).reset_index(drop=True)

# Change Sponsor

# Croatia -> Change to P4
df_al.loc[df_al.TADIG_CODE_AL.isin(['HRVT2','HRVVI']),'SPONSOR'] = 'S2'

# Russian Federation -> Change to Partner
df_al.loc[df_al.TADIG_CODE_AL.isin(['RUSBD','RUS01','RUSNW']),'SPONSOR'] = 'S1'

# Turkey -> Change to P4
df_al.loc[df_al.TADIG_CODE_AL.isin(['TURAC','TURIS','TURTC','TURTK']),'SPONSOR'] = 'S2'
df_al.drop(df_al[df_al.TADIG_CODE_AL=='TURTS'].index,inplace=True)

print('Allowed list:')
display(df_al.head())

Allowed list:


,NETWORK_ID,SPONSOR,TADIG_CODE_AL
0,622.0,S2,MNEMT
1,569.0,S1,MKDCC
2,234.0,S2,CZEET
3,577.0,S1,MWICT
4,439.0,S1,IDNTS


In [143]:
# Объединяем тарифы с TADIG и Allowed list

# Результирующий файл с тарифами Partner и P4 и укащаеием принадлежности в Combined

file_partner_p4_compare = 'Partner_P4_Compare_1_1_2019.csv'

# Неизвестные TADIG в тарифах Partner с добавлением network_id
df_partner_net_id = pd.merge(df_partner,df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
print('Неизвестные TADIG в тарифах Partner с добавлением network_id:')
display(df_partner_net_id[df_partner_net_id.TADIG_CODE.isnull()])

# Неизвестные TADIG в тарифах P4 с добавлением network_id
df_p4_net_id = pd.merge(df_p4,df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
print('Неизвестные TADIG в тарифах P4 с добавлением network_id:')
display(df_p4_net_id[df_p4_net_id.TADIG_CODE.isnull()])

# Тарифы Partner и P4
df_s1_s2_temp = pd.merge(df_partner_net_id[['MOC','MTC','SMS','MB','NETWORK_ID','Voice_Increment_SEC','Data_Increment_KB']],
                    df_p4_net_id[['MOC','MTC','SMS','MB','NETWORK_ID','Voice_Increment_SEC','Data_Increment_KB']],
                    how='outer',on=['NETWORK_ID'],suffixes=('_S1','_S2')).drop_duplicates()

# Тарифы Partner и P4 с TADIG
df_s1_s2 = pd.merge(df_s1_s2_temp,df_tadig,how='left',on='NETWORK_ID')

# Тарифы Partner и P4 с привязкой к Allowed list
df_s1_s2_sponsor = pd.merge(df_s1_s2,df_al,how='outer',on='NETWORK_ID')

# Результирующий файл
# Тарифы Partner и P4 с привязкой к Allowed list

#f_list = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MOC_S1','MOC_S2','MTC_S1','MTC_S2','SMS_S1','SMS_S2','MB_S1','MB_S2','SPONSOR']
f_list = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE',
          'MOC_S1','MTC_S1','SMS_S1','MB_S1','Voice_Increment_SEC_S1','Data_Increment_KB_S1',
          'MOC_S2','MTC_S2','SMS_S2','MB_S2','Voice_Increment_SEC_S2','Data_Increment_KB_S2',
          'SPONSOR','TADIG_CODE_AL','NETWORK_ID']
df_s1_s2_sponsor = df_s1_s2_sponsor[f_list].drop_duplicates()

#df_s1_s2_sponsor.to_csv(join(load_folder,file_partner_p4_compare),index=False)

df_s1_s2_sponsor.head()

Неизвестные TADIG в тарифах Partner с добавлением network_id:


,TADIG,MCCMNC,PARTNER,COUNTRY,Operator,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
86,INDEU,40456.0,IdeaUP IN,India,IDEA,0.1,0.0,0.06,0.02,60,10.0,NaN,NaN,NaN,NaN
87,INDIB,40570.0,IDEA Bih IN,India,IDEA,0.1,0.0,0.06,0.02,60,10.0,NaN,NaN,NaN,NaN


Неизвестные TADIG в тарифах P4 с добавлением network_id:


,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
137,Mozambique,MOVITEL S.A,MOZVG,0.7,NaN,0.07,0.042,60,10,NaN,NaN,NaN,NaN


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S1,MTC_S1,SMS_S1,MB_S1,Voice_Increment_SEC_S1,Data_Increment_KB_S1,MOC_S2,MTC_S2,SMS_S2,MB_S2,Voice_Increment_SEC_S2,Data_Increment_KB_S2,SPONSOR,TADIG_CODE_AL,NETWORK_ID
0,Albania,Vodafone,ALBVF,0.0439,0.0,0.0062,0.0030,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,S1,ALBVF,12.0
1,Andorra,Andorra Telecom S A U,ANDMA,0.2000,0.0,0.0300,0.2000,60.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0
2,Anguilla,Cable and Wireless,AIACW,0.1606,0.0,0.0128,0.0257,60.0,1.0,0.8,NaN,0.08,0.08,1.0,1.0,S1,AIACW,20.0
3,Antigua and Barbuda,C & W,ATGCW,0.1500,0.0,0.0120,0.0240,60.0,1.0,0.8,NaN,0.08,0.08,1.0,1.0,S1,ATGCW,23.0
4,Argentina,Movistar/Telefonica,ARG01,0.0728,0.0,0.0240,0.0180,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,S1,ARG01,27.0


In [142]:
# Проверка наличия различных тарифов для одного и того же оператора.
# Операторы определяются по данным OCS

df = df_p4_net_id.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s1 = df[df.values > 1]
display(s1)
df = df_partner_net_id.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s1 = df[df.values > 1]
display(s1)
#df_s1_s2_sponsor[df_s1_s2_sponsor.NETWORK_ID.isin(s1.index.values)]

,MOC,MTC,SMS,MB
NETWORK_ID,,,,


,MOC,MTC,SMS,MB
NETWORK_ID,,,,


In [149]:
# Подготовка файла тарифов для загрузки в базу данных OCS для Partner

# Файлы:
# Переход disc -> Not disc для отправки спонсору для запроса тарифов
# Для генерации таблицы в БД

file_s1_disc_to_not_disc = 'df_s1_disc_to_not_disc.csv'
file_s1_to_db = 'df_s1_insert_267.csv'
file_s1_to_db_100 = 'df_s1_insert_267_100.csv'

# Объединение новых тарифов со старыми, выгруженными из базы данных
df_temp = pd.merge(df_s1_s2_sponsor.loc[df_s1_s2_sponsor.MB_S1.notnull(),
                                        ['NETWORK_ID','MOC_S1','MTC_S1','SMS_S1','MB_S1']],
         df_roaming_plan_rules_s1[['ROAMING_PLAN_RULE_ID','NETWORK_ID','is_discounted',
                                   'MO_CALL_RATE', 'MT_CALL_RATE', 'MO_SMS_RATE', 'MT_SMS_RATE','DATA_RATE']],
         how='outer',on='NETWORK_ID').drop_duplicates()

# New
df = df_temp[df_temp.is_discounted.isnull()].copy()
df['com'] = 'new'
df_s1_to_db = df[['ROAMING_PLAN_RULE_ID','NETWORK_ID','MOC_S1','MTC_S1','SMS_S1','MB_S1','is_discounted','com']].copy()

# Not disc -> disc
df = df_temp[(df_temp.is_discounted == False) & (df_temp.MB_S1.notnull())].copy()
df['com'] = 'Not disc -> disc'
df_s1_to_db = df_s1_to_db.append(df[['ROAMING_PLAN_RULE_ID','NETWORK_ID','MOC_S1','MTC_S1','SMS_S1','MB_S1','is_discounted','com']])

# Not disc -> Not disc
df = df_temp[(df_temp.is_discounted == False) & (df_temp.MB_S1.isnull())].copy()
df.drop(['MOC_S1','MTC_S1','SMS_S1','MB_S1'],axis=1,inplace=True)
df.rename({'MO_CALL_RATE':'MOC_S1','MT_CALL_RATE':'MTC_S1','MO_SMS_RATE':'SMS_S1','DATA_RATE':'MB_S1'},axis=1,inplace=True)
df['com'] = 'Not disc -> Not disc'
df_s1_to_db = df_s1_to_db.append(df[['ROAMING_PLAN_RULE_ID','NETWORK_ID','MOC_S1','MTC_S1','SMS_S1','MB_S1','is_discounted','com']])

# disc -> Not disc
df = df_temp[(df_temp.is_discounted == True) & (df_temp.MB_S1.isnull())].copy()
df['com'] = 'disc -> Not disc'
df_s1_to_db = df_s1_to_db.append(df[['ROAMING_PLAN_RULE_ID','NETWORK_ID','MOC_S1','MTC_S1','SMS_S1','MB_S1','is_discounted','com']])
df = pd.merge(df,df_tadig,how='left',on='NETWORK_ID')\
[['TADIG_CODE','ROAMING_PLAN_RULE_ID','NETWORK_ID','MOC_S1','MTC_S1','SMS_S1','MB_S1','is_discounted','com']]
df.to_csv(join(load_folder,file_s1_disc_to_not_disc),index=False)
print('disc -> Not disc:')
display(df)

# disc -> disc
df = df_temp[(df_temp.is_discounted == True) & (df_temp.MB_S1.notnull())].copy()
df['com'] = 'disc -> disc'
df_s1_to_db = df_s1_to_db.append(df[['ROAMING_PLAN_RULE_ID','NETWORK_ID','MOC_S1','MTC_S1','SMS_S1','MB_S1','is_discounted','com']])

# Добавляем поля для удобства выгрузки
#!!! Поменять 'CURRENCY_ID', 'SPONSOR_ID' и 'START_DATE'

#df_s1_to_db['ROAMING_PLAN_RULE_ID']=''
df_s1_to_db['ROAMING_PLAN_ID']=''
df_s1_to_db['SPONSOR_ID']=1
df_s1_to_db['MT_SMS_RATE']=0
df_s1_to_db['CURRENCY_ID']=1 #1:USD, 2:EUR
df_s1_to_db['START_DATE']='2019-01-01 00:00:00'
df_s1_to_db['END_DATE']='NULL'

df_s1_to_db = pd.merge(df_s1_to_db,df_tadig[['TADIG_CODE','NETWORK_ID']],how='left',on='NETWORK_ID')

df_s1_to_db_100 = df_s1_to_db.copy()
df_s1_to_db_100[['MOC_S1','MTC_S1','SMS_S1','MB_S1']] = df_s1_to_db_100[['MOC_S1','MTC_S1','SMS_S1','MB_S1']]*2

df_s1_to_db[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC_S1',
'MTC_S1',
'SMS_S1',
'MT_SMS_RATE',
'MB_S1',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted',
'com']].drop_duplicates().to_csv(join(load_folder,file_s1_to_db),index=False)

df_s1_to_db_100[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC_S1',
'MTC_S1',
'SMS_S1',
'MT_SMS_RATE',
'MB_S1',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted',
'com']].drop_duplicates().to_csv(join(load_folder,file_s1_to_db_100),index=False)

disc -> Not disc:


,TADIG_CODE,ROAMING_PLAN_RULE_ID,NETWORK_ID,MOC_S1,MTC_S1,SMS_S1,MB_S1,is_discounted,com
0,CANMC,13733.0,156.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
1,CMRVT,13741.0,146.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
2,DEUE1,13753.0,328.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
3,ESPT2,13762.0,1147.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
4,GHAZN,13778.0,344.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
5,GUYUM,13784.0,377.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
6,HKGNW,13788.0,391.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
7,HTIHT,13791.0,380.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
8,MKDCC,13852.0,569.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
9,MWICT,13862.0,577.0,NaN,NaN,NaN,NaN,True,disc -> Not disc


In [159]:
# Подготовка файла тарифов для загрузки в базу данных OCS для P4

# Файлы:
# Переход disc -> Not disc для отправки спонсору для запроса тарифов
# Для генерации таблицы в БД

file_s2_disc_to_not_disc = 'df_s2_disc_to_not_disc.csv'
file_s2_to_db = 'df_s2_inser_268.csv'
file_s2_to_db_100 = 'df_s2_insert_100.csv'


df_temp = pd.merge(df_s1_s2_sponsor.loc[df_s1_s2_sponsor.MB_S2.notnull(),
                                        ['NETWORK_ID','MOC_S2','MTC_S2','SMS_S2','MB_S2']],
         df_roaming_plan_rules_s2[['ROAMING_PLAN_RULE_ID','NETWORK_ID','is_discounted',
                                   'MO_CALL_RATE', 'MT_CALL_RATE', 'MO_SMS_RATE', 'MT_SMS_RATE','DATA_RATE']],
         how='outer',on='NETWORK_ID').drop_duplicates()

# New
df = df_temp[df_temp.is_discounted.isnull()].copy()
df['com'] = 'new'
df_s2_to_db = df[['ROAMING_PLAN_RULE_ID','NETWORK_ID','MOC_S2','MTC_S2','SMS_S2','MB_S2','is_discounted','com']].copy()

# Not disc -> disc
df = df_temp[(df_temp.is_discounted == False) & (df_temp.MB_S2.notnull())].copy()
df['com'] = 'Not disc -> disc'
df_s2_to_db = df_s2_to_db.append(df[['ROAMING_PLAN_RULE_ID','NETWORK_ID','MOC_S2','MTC_S2','SMS_S2','MB_S2','is_discounted','com']])

# Not disc -> Not disc
df = df_temp[(df_temp.is_discounted == False) & (df_temp.MB_S2.isnull())].copy()
df.drop(['MOC_S2','MTC_S2','SMS_S2','MB_S2'],axis=1,inplace=True)
df.rename({'MO_CALL_RATE':'MOC_S2','MT_CALL_RATE':'MTC_S2','MO_SMS_RATE':'SMS_S2','DATA_RATE':'MB_S2'},axis=1,inplace=True)
df['com'] = 'Not disc -> Not disc'
df_s2_to_db = df_s2_to_db.append(df[['ROAMING_PLAN_RULE_ID','NETWORK_ID','MOC_S2','MTC_S2','SMS_S2','MB_S2','is_discounted','com']])

# disc -> Not disc
df = df_temp[(df_temp.is_discounted == True) & (df_temp.MB_S2.isnull())].copy()
df['com'] = 'disc -> Not disc'
df_s2_to_db = df_s2_to_db.append(df[['ROAMING_PLAN_RULE_ID','NETWORK_ID','MOC_S2','MTC_S2','SMS_S2','MB_S2','is_discounted','com']])
df = pd.merge(df,df_tadig,how='left',on='NETWORK_ID')\
[['TADIG_CODE','ROAMING_PLAN_RULE_ID','NETWORK_ID','MOC_S2','MTC_S2','SMS_S2','MB_S2','is_discounted','com']]
df.to_csv(join(load_folder,file_s2_disc_to_not_disc),index=False)
display(df.head())

# disc -> disc
df = df_temp[(df_temp.is_discounted == True) & (df_temp.MB_S2.notnull())].copy()
df['com'] = 'disc -> disc'
df_s2_to_db = df_s2_to_db.append(df[['ROAMING_PLAN_RULE_ID','NETWORK_ID','MOC_S2','MTC_S2','SMS_S2','MB_S2','is_discounted','com']])

#Добавляем поля для удобства выгрузки
#!!! Поменять 'ROAMING_PLAN_ID', 'CURRENCY_ID', 'SPONSOR_ID' и 'START_DATE'

#df_s2_to_db['ROAMING_PLAN_RULE_ID']=''
df_s2_to_db['ROAMING_PLAN_ID']='268'
df_s2_to_db['SPONSOR_ID']=2
df_s2_to_db['MT_SMS_RATE']=0
df_s2_to_db['CURRENCY_ID']=2 #1:USD, 2:EUR
df_s2_to_db['START_DATE']='2019-01-01 00:00:00'
df_s2_to_db['END_DATE']='NULL'

df_s2_to_db = pd.merge(df_s2_to_db,df_tadig[['TADIG_CODE','NETWORK_ID']],how='left',on='NETWORK_ID')

df_s2_to_db_100 = df_s2_to_db.copy()
df_s2_to_db_100[['MOC_S2','MTC_S2','SMS_S2','MB_S2']] = df_s2_to_db_100[['MOC_S2','MTC_S2','SMS_S2','MB_S2']]*2

df_s2_to_db_100.loc[df_s2_to_db_100.NETWORK_ID.isin(['1100','1036']),['MOC_S2','MTC_S2','SMS_S2','MB_S2']] = \
df_s2_to_db_100.loc[df_s2_to_db_100.NETWORK_ID.isin(['1100','1036']),['MOC_S2','MTC_S2','SMS_S2','MB_S2']]*1.25

df_s2_to_db[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC_S2',
'MTC_S2',
'SMS_S2',
'MT_SMS_RATE',
'MB_S2',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted',
'com']].drop_duplicates().to_csv(join(load_folder,file_s2_to_db),index=False)

df_s2_to_db_100[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC_S2',
'MTC_S2',
'SMS_S2',
'MT_SMS_RATE',
'MB_S2',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted',
'com']].drop_duplicates().to_csv(join(load_folder,file_s2_to_db_100),index=False)

,TADIG_CODE,ROAMING_PLAN_RULE_ID,NETWORK_ID,MOC_S2,MTC_S2,SMS_S2,MB_S2,is_discounted,com
0,AUTMM,4378.0,51.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
1,AUTTR,4378.0,51.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
2,HRVCN,4395.0,224.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
3,CZERM,4402.0,235.0,NaN,NaN,NaN,NaN,True,disc -> Not disc
4,FRAFM,4418.0,296.0,NaN,NaN,NaN,NaN,True,disc -> Not disc


In [169]:
# Подготовка файла тарифов для загрузки в базу данных OCS для Real Combined Price Plan

# Файлы:
file_combined_100 = 'df_combined_100.csv'

df1 = df_s1_s2_sponsor[df_s1_s2_sponsor.SPONSOR=='S1']\
[['TADIG_CODE','MOC_S1','MTC_S1','SMS_S1','MB_S1','SPONSOR','NETWORK_ID','TADIG_CODE_AL']].copy()

df1.rename({'TADIG_CODE':'TADIG','MOC_S1':'MOC','MTC_S1':'MTC','SMS_S1':'SMS','MB_S1':'DATA'},axis=1,inplace=True)

#Добавляем поля для удобства выгрузки
#!!! Поменять 'ROAMING_PLAN_ID', 'CURRENCY_ID', 'SPONSOR_ID' и 'START_DATE'

df1['ROAMING_PLAN_RULE_ID']=''
df1['ROAMING_PLAN_ID']=''
df1['SPONSOR_ID']=1
df1['MT_SMS_RATE']=0
df1['CURRENCY_ID']=1 #1:USD, 2:EUR
df1['START_DATE']='2019-01-01 00:00:00'
df1['END_DATE']='NULL'
df1['is_discounted']=1

df2 = df_s1_s2_sponsor[df_s1_s2_sponsor.SPONSOR=='S2']\
[['TADIG_CODE','MOC_S2','MTC_S2','SMS_S2','MB_S2','SPONSOR','NETWORK_ID','TADIG_CODE_AL']].copy()

df2.rename({'TADIG_CODE':'TADIG','MOC_S2':'MOC','MTC_S2':'MTC','SMS_S2':'SMS','MB_S2':'DATA'},axis=1,inplace=True)

#Добавляем поля для удобства выгрузки
#!!! Поменять 'ROAMING_PLAN_ID', 'CURRENCY_ID', 'SPONSOR_ID' и 'START_DATE'

df2['ROAMING_PLAN_RULE_ID']=''
df2['ROAMING_PLAN_ID']=''
df2['SPONSOR_ID']=2
df2['MT_SMS_RATE']=0
df2['CURRENCY_ID']=2 #1:USD, 2:EUR
df2['START_DATE']='2019-01-01 00:00:00'
df2['END_DATE']='NULL'
df2['is_discounted']=1

df_combined = pd.concat([df1, df2],axis=0).drop_duplicates()
df_combined.MTC.fillna(0,inplace=True)
df_combined.dropna(inplace=True)


df_combined_100 = df_combined.copy()
df_combined_100[['MOC','MTC','SMS','DATA']] = df_combined_100[['MOC','MTC','SMS','DATA']]*2

df_combined_100.loc[df_combined_100.NETWORK_ID.isin(['1100','1036']),['MOC','MTC','SMS','DATA']] = \
df_combined_100.loc[df_combined_100.NETWORK_ID.isin(['1100','1036']),['MOC','MTC','SMS','DATA']]*1.25

df_combined_100[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC',
'MTC',
'SMS',
'MT_SMS_RATE',
'DATA',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted',
'TADIG_CODE_AL']].drop_duplicates().to_csv(join(load_folder,file_combined_100),index=False)

In [181]:
# Ищем в Allowed lists 'Partner_Combined' и 'P4_Combined' что нужно убрать/добавить

# Создаем файл

file_combined_al = 'allowed_list.csv'

df = df_combined_100[['NETWORK_ID','SPONSOR_ID']].drop_duplicates().copy()

df_al_=pd.read_csv(join(load_folder,file_allowed_list),usecols=['Sponsor','PLMN code'])
df_al_.rename({'PLMN code':'TADIG_CODE_AL','Sponsor':'SPONSOR'},inplace=True,axis=1)
df_al_ = df_al_[df_al_.SPONSOR.isin(['Partner_Combined','P4_Combined'])]
df_al_ = df_al_.replace({'Partner_Combined':'S1','P4_Combined':'S2'})
df_al_ = pd.merge(df_al_,df_tadig[['TADIG_CODE','NETWORK_ID']],how='left',left_on='TADIG_CODE_AL',right_on='TADIG_CODE')
df_al_.drop('TADIG_CODE',inplace=True,axis=1)

pd.merge(df_al_,df,how='outer',on='NETWORK_ID').to_csv(join(load_folder,file_combined_al),index=False)

In [ ]:
# Подготовка файлов для модификации существующих Combine PP



In [67]:
df_s1_s2_sponsor.to_csv(join(load_folder,'result.csv'),index=False)